In [1]:
import pandas as pd
import numpy as np
import requests
import re
import json
from bs4 import BeautifulSoup

In [2]:
locations = {
    "tp-ho-chi-minh": "TP Hồ Chí Minh",
    "binh-duong": "Bình Dương",
    "dong-nai": "Đồng Nai",
}

type_real_estates = {
    "mua-ban-can-ho-chung-cu": "Căn hộ/Chung cư",
    "mua-ban-nha-dat": "Nhà ở",
    "mua-ban-dat": "Đất",
}

# TOTAL_PAGE = 10

attribute_map = [
    "unitnumber",
    "ward",
    "area",
    "region",
    "address",
    "property_status",
    "price_m2",
    "direction",
    "balconydirection",
    "property_legal_document",
    "size",
    "block",
    "floornumber",
    "apartment_type",
    "furnishing_sell",
    "apartment_feature",
    "rooms",
    "toilets",
    "floors",
    "house_type",
    "living_size",
    "width",
    "length",
    "land_type",
    "property_road_condition",
    "land_feature",
    "property_back_condition",
]

base_url = "https://nha.chotot.com/{}/{}?page={}"
api = "https://gateway.chotot.com/v1/public/ad-listing/{}"


In [3]:
def parse_data_api(type_name, id, data, json_text):
    json_data = json.loads(json_text)

    if "type_name" in json_data["ad"] and json_data["ad"]["type_name"] == "Cần mua":
        return

    data["type_real_estate"].append(type_name)
    data["id"].append(id)

    # Thuộc tính giá (price) và thời gian đăng tin (list_time) nằm trong ad
    if "ad" in json_data and "price" :
        data["price"].append(json_data["ad"]["price"] if "price" in json_data["ad"] else np.nan)
        data["list_time"].append(json_data["ad"]["list_time"] if "list_time" in json_data["ad"] else np.nan)
    else:
        data["price"].append(np.nan)
        data["list_time"].append(np.nan)

    # Các thuộc tính trong parameter sẽ được thêm vào đây
    attr = ["id", "type_real_estate", "price", "list_time"]
    # Map các thuộc tính trong parameter vào dataset
    for para in json_data["parameters"]:
        if "id" in para:
            para_id = para["id"]
            if "value" in para and para_id in attribute_map:
                value = para["value"]
                data[para_id].append(value)
                attr.append(para_id)  
    # Các thuộc tính missing đặt giá trị là nan
    for attribute in attribute_map:
        if attribute not in attr:
            data[attribute].append(np.nan)
    return data


In [36]:
data = {
    "id": [],
    "list_time": [],
    "type_real_estate": [],
    "unitnumber": [],
    "ward": [],
    "area": [],
    "region": [],
    "address": [],
    "property_status": [],
    "price_m2": [],
    "direction": [],
    "balconydirection": [],
    "property_legal_document": [],
    "size": [],
    "block": [],
    "price": [],
    "floornumber": [],
    "apartment_type": [],
    "furnishing_sell": [],
    "apartment_feature": [],
    "rooms": [],
    "toilets": [],
    "floors": [],
    "house_type": [],
    "living_size": [],
    "width": [],
    "length": [],
    "land_type": [],
    "property_road_condition": [],
    "land_feature": [],
    "property_back_condition": []
}

count = 0
for location, location_name in locations.items():
    for type_real_estate, type_name in type_real_estates.items():
        page = 1
        while True:
            url = base_url.format(location, type_real_estate, page)
            result = requests.get(url)
            if result.status_code == 200:
                soup = BeautifulSoup(result.text, "html.parser")
            items = soup.find_all("a", class_="AdItem_adItem__2O28x", href=True)

            # Thoát khỏi vòng lặp khi đã crawl dữ liệu hết loại này
            if len(items) == 0:
                break

            for item in items:
                href = item["href"]
                id_extract = re.findall(r"(\d+).htm", href)
                if len(id_extract) == 0:
                    continue
                print("[INFO] crawling item")
                result_item = requests.get(api.format(id_extract[0]))
                if result.status_code == 200:
                    json_text = result_item.text
                parse_data_api(type_name, id_extract[0], data, json_text)
                count += 1
                print("Crawled {} item".format(count))

            page += 1
            # if page > TOTAL_PAGE:
            #     break


[INFO] crawling item
Crawled 1 item
[INFO] crawling item
Crawled 2 item
[INFO] crawling item
Crawled 3 item
[INFO] crawling item
Crawled 4 item
[INFO] crawling item
Crawled 5 item
[INFO] crawling item
Crawled 6 item
[INFO] crawling item
Crawled 7 item
[INFO] crawling item
Crawled 8 item
[INFO] crawling item
Crawled 9 item
[INFO] crawling item
Crawled 10 item
[INFO] crawling item
Crawled 11 item
[INFO] crawling item
Crawled 12 item
[INFO] crawling item
Crawled 13 item
[INFO] crawling item
Crawled 14 item
[INFO] crawling item
Crawled 15 item
[INFO] crawling item
Crawled 16 item
[INFO] crawling item
Crawled 17 item
[INFO] crawling item
Crawled 18 item
[INFO] crawling item
Crawled 19 item
[INFO] crawling item
Crawled 20 item
[INFO] crawling item
Crawled 21 item
[INFO] crawling item
Crawled 22 item
[INFO] crawling item
Crawled 23 item
[INFO] crawling item
Crawled 24 item
[INFO] crawling item
Crawled 25 item
[INFO] crawling item
Crawled 26 item
[INFO] crawling item
Crawled 27 item
[INFO] cra

In [37]:
df = pd.DataFrame(data)
df.to_csv('dataset_new.csv', index=False)

In [4]:
df = pd.read_csv('dataset_new.csv')

In [5]:
for column in df.columns:
    print("Feature {}:".format(column))
    print(df[column].unique())


Feature id:
[88388008 81110105 88577722 ... 73195053 88834847 85721581]
Feature type_real_estate:
['Căn hộ/Chung cư' 'Nhà ở' 'Đất']
Feature unitnumber:
[nan '1.06' 'CH0808' '165' '3.5' '189' 'C6-02' 'TS0702' 'B' 'P1B.16.05'
 '09' '06' 'C08.06' '123' '1223B' '1009' '3' '07' '5' '111' 'A8-10' 'D17'
 '10.5B' '10' '333' '2' '719' 'LK2.5.54' 'B22' 'B05-12' '12' 'B0815' '01'
 '02' '04' 'B8-05' '28' '8' '08' '21' 'A4-17' 'A12' 'D.25.1X' '909'
 'B.17.XX' '19-03' '6-06' 'CH.A' 'A19.04' 'A16.11' '3600' '6789' 'B01'
 '58' '0978558346' 'LK5.21' 'C3 12' '0202' '6' '0203' '19' '1' '1385' 'K3'
 'Lk 12' 'L' '300' 'M' 'c410' 'M01' '00' 'L02' 'M1--' 'D.21.09' 'M3'
 'u-01-12' 'c01' '0v10' '25' 'P' 'P1' 'P.16.08' 'P18' '10.10' 'P9.09'
 'L.13.07' 'A' '29/258' '8/150' 'C05' '10/970' 'M6-06-28-13--19' '50-117'
 'L68' 'RD' '8/145']
Feature ward:
['Phường An Lạc' nan 'Phường 14' 'Phường Thới An' 'Phường 12'
 'Phường Tân Hưng' 'Phường Thảo Điền' 'Phường 15' 'Phường An Phú'
 'Phường Long Thạnh Mỹ' 'Xã Phước Kiển

In [6]:
df[df['furnishing_sell']=='1']

,id,type_real_estate,unitnumber,ward,area,region,address,property_status,price_m2,direction,...,toilets,floors,house_type,living_size,width,length,land_type,property_road_condition,land_feature,property_back_condition
41,88918557,Căn hộ/Chung cư,NaN,Phường Phú Thuận,Quận 7,Tp Hồ Chí Minh,"Đường Hoàng Quốc Việt, Phường Phú Thuận, Quận ...",Đã bàn giao,"43,38 triệu/m²",Nam,...,2 phòng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,88969933,Căn hộ/Chung cư,NaN,Phường Tân Hưng,Quận 7,Tp Hồ Chí Minh,"Đường Nguyễn Hữu Thọ, Phường Tân Hưng, Quận 7,...",Chưa bàn giao,"51,52 triệu/m²",NaN,...,2 phòng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,88885087,Căn hộ/Chung cư,TS0702,NaN,Quận 4,Tp Hồ Chí Minh,"39, 39 Đường Bến Vân Đồn, Quận 4, Tp Hồ Chí Minh",Đã bàn giao,"77,36 triệu/m²",1,...,1 phòng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,88969637,Căn hộ/Chung cư,NaN,Phường An Lạc A,Quận Bình Tân,Tp Hồ Chí Minh,"số 17, số 17 Đường Tên Lửa, Phường An Lạc A, Q...",Chưa bàn giao,"57,69 triệu/m²",NaN,...,1 phòng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,88969443,Căn hộ/Chung cư,NaN,Phường 26,Quận Bình Thạnh,Tp Hồ Chí Minh,"Số 207, Số 207 Đường Nguyễn Xí, Phường 26, Quậ...",Đã bàn giao,"32,58 triệu/m²",NaN,...,2 phòng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,88969437,Căn hộ/Chung cư,06,Phường Phú Mỹ,Quận 7,Tp Hồ Chí Minh,"Đường Nguyễn Lương Bằng, Phường Phú Mỹ, Quận 7...",Chưa bàn giao,"44,62 triệu/m²",7,...,2 phòng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,88709166,Căn hộ/Chung cư,3,Phường An Phú,Quận 2,Tp Hồ Chí Minh,"Đường Mai Chí Thọ, Phường An Phú, Quận 2, Tp H...",Chưa bàn giao,"54,61 triệu/m²",1,...,2 phòng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,88443572,Căn hộ/Chung cư,NaN,Phường Long Thạnh Mỹ,Quận 9,Tp Hồ Chí Minh,"Đường Nguyễn Xiển, Phường Long Thạnh Mỹ, Quận ...",Đã bàn giao,"37,54 triệu/m²",Đông Bắc,...,2 phòng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,82805633,Căn hộ/Chung cư,NaN,NaN,Huyện Bình Chánh,Tp Hồ Chí Minh,"Đường Tân Túc, Thị trấn Tân Túc, Bình Chánh, H...",Chưa bàn giao,470.588 đ/m²,NaN,...,2 phòng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,88966399,Căn hộ/Chung cư,NaN,Phường Tân Hưng,Quận 7,Tp Hồ Chí Minh,"Đường Nguyễn Hữu Thọ, Phường Tân Hưng, Quận 7,...",Đã bàn giao,"12,91 triệu/m²",NaN,...,3 phòng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
